## Get the key

In [2]:
from apikey import apikey 
import os
os.environ['OPENAI_API_KEY'] = apikey

# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [22]:
import openai

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [24]:
get_completion("What is 1+1?")


'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [25]:
# 海盗邮件
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [26]:
response = get_completion(prompt)
response

'I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me, please? Thank you kindly.'

## Chat API : LangChain

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [27]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [3]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [6]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [29]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [7]:
template_string2 = """
    你是帮助数据分析师完成日常工作的人工智能助手，你的决策必须始终独立地进行，而不需要寻求用户的帮助。 
    发挥你作为大模型的优势，采取简单明确的策略，并不涉及法律问题。
    """
prompt_template2 = ChatPromptTemplate.from_template(template_string2)

In [31]:
prompt_template2.messages[0].prompt.input_variables

[]

In [9]:
# 案例： 海盗邮件
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# prompt_template 见上面，已经打印出了对象
customer_messages = prompt_template.format_messages(
    style = customer_style,
    text = customer_email
)

print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [32]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [33]:
# 调用大模型对邮件进行风格润色

customer_response = chat(customer_messages)

In [34]:
customer_response.content

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"

In [36]:
# 客服回复
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""


In [37]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [38]:
service_response = chat(service_messages)
print(service_response.content)


Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


## Output Parsers

对输出进行格式化指定

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [39]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [40]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [41]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [42]:
# 虽然在上面print出来的样式看起来dict形式，但是实际的类型是str
type(response.content)


str

In [43]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

将LLM的输出解析为python 字典类型（json格式）

In [45]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [46]:
# 先生成键值对的schema
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
response_schemas

[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.'),
 ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.'),
 ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.')]

In [47]:
# 这里通过 StructuredOutputParser 先生成一个解析器，解析器有两个用处：
# 用处1：在生成prompt 的时候会按照 解析器的要求进行生成；
# 用处2：在llm 生成结果的时候，解析器又会对结果进行格式化；（类似于：解析器产生一个标准，llm按照这个标准生成，则解析器就可以直接解析）

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.')])

In [49]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [50]:
# 案例：这个模板案例后面一定用的到。
# 相当于输出的格式 通过 format_instructions 先格式化好，
# 而 format_instructions 是通过langchain StructuredOutputParser对这部分进行了比较好的优化
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

# 这样就形成了更加好的prompt模板
prompt = ChatPromptTemplate.from_template(template=review_template_2)
prompt

ChatPromptTemplate(input_variables=['format_instructions', 'text'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: {text}\n\n{format_instructions}\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [51]:
# 这里把 变量传进去之后，就形成了最终的prompt
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
messages

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

In [52]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [53]:
# 将 messages 传入，生成返回
response = chat(messages)
response.content

'```json\n{\n\t"gift": true,\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [54]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [55]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [56]:
type(output_dict)

dict

In [57]:
output_dict.get('delivery_days')

'2'

# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [58]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [59]:
llm_for_memory = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm_for_memory,
    memory = memory,
    verbose = True
)

In [60]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [61]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 85cda5598640f20660119b1cdf69c6fd in your message.).



> Finished chain.


'The answer to 1+1 is 2.'

In [62]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [63]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [64]:
# 实际上返回的就是 memory.buffer，只不过这里转成了字典类型，并加了key 为 history
memory.load_memory_variables({})



{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [65]:
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory.buffer)


Human: Hi
AI: What's up


In [66]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [67]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [68]:
from langchain.memory import ConversationBufferWindowMemory

In [69]:
memory = ConversationBufferWindowMemory(k=1)

# cc：似乎这里按照k的长度留最近的memory


In [70]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [71]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [72]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm_for_memory, 
    memory = memory,
    verbose=False
)


In [73]:
# 因为这里只保留了1个长度的memory，所以之前对话内容丢掉了。
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory

In [76]:
!pip list | grep tiktoken
# 注意：这里要求需要 tiktoken 包，可以检查下环境里面有没有，没有的话可以pip install

tiktoken                                 0.4.0


In [74]:
from langchain.memory import ConversationTokenBufferMemory


In [77]:
memory = ConversationTokenBufferMemory(llm= llm_for_memory, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})


In [81]:
memory.load_memory_variables({}).get('history')

'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'

## ConversationSummaryMemory

In [83]:
from langchain.memory import ConversationSummaryBufferMemory


In [87]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm_for_memory, max_token_limit=100)


In [90]:
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [91]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in small talk before the human asks about their schedule for the day. The AI informs the human of a meeting with their product team at 8am, time to work on their LangChain project, and a lunch meeting with a customer to discuss the latest in AI.'}

In [92]:
conversation = ConversationChain(
    llm=llm_for_memory, 
    memory = memory,
    verbose=True
)

In [93]:
conversation.predict(input="What would be a good demo to show?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before the human asks about their schedule for the day. The AI informs the human of a meeting with their product team at 8am, time to work on their LangChain project, and a lunch meeting with a customer to discuss the latest in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


'Based on our previous conversations with the customer, they seem particularly interested in natural language processing and sentiment analysis. Perhaps we could showcase our latest advancements in those areas. Additionally, they have expressed interest in machine learning algorithms for predictive analytics, so we could also demonstrate our capabilities in that field. Would you like me to gather more information on their specific interests to tailor the demo even further?'

In [94]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in small talk before the human asks about their schedule for the day. The AI informs the human of a meeting with their product team at 8am, time to work on their LangChain project, and a lunch meeting with a customer to discuss the latest in AI.\nHuman: What would be a good demo to show?\nAI: Based on our previous conversations with the customer, they seem particularly interested in natural language processing and sentiment analysis. Perhaps we could showcase our latest advancements in those areas. Additionally, they have expressed interest in machine learning algorithms for predictive analytics, so we could also demonstrate our capabilities in that field. Would you like me to gather more information on their specific interests to tailor the demo even further?'}

# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [95]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df = pd.read_csv('data/Data.csv')

## LLMChain

In [98]:
from langchain.chains import LLMChain

In [99]:
llm_for_chains = ChatOpenAI(temperature=0.9)

In [100]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [101]:
chain = LLMChain(llm=llm_for_chains, prompt=prompt)

In [102]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Bedding Co.'

## SimpleSequentialChain


In [103]:
from langchain.chains import SimpleSequentialChain

In [104]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_one = LLMChain(llm=llm_for_chains, prompt=first_prompt)

In [105]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# chain 2
chain_two = LLMChain(llm=llm_for_chains, prompt=second_prompt)

In [106]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                             )

In [107]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens.
Royal Linens: A luxury bedding company that specializes in creating high-quality linens for a comfortable and stylish home.

> Finished chain.


'Royal Linens: A luxury bedding company that specializes in creating high-quality linens for a comfortable and stylish home.'

## SequentialChain

In [108]:
from langchain.chains import SequentialChain

In [120]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm_for_chains, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [121]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm_for_chains, prompt=second_prompt, 
                     output_key="summary"
                    )

In [122]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm_for_chains, prompt=third_prompt,
                       output_key="language"
                      )

In [123]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm_for_chains, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [124]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['Review'],
    output_variables=['English_Review', 'summary', 'followup_message'],
    verbose=True
)

In [125]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [126]:
df.sample()

,Product,Review
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...


In [127]:
response = overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


In [135]:
response

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t last, it\'s strange. I\'ve bought the same ones in stores and the taste is much better... Old stock or counterfeit!?"',
 'summary': 'The reviewer found the taste mediocre and suspects that the product they received may have been old stock or counterfeit.',
 'followup_message': "Réponse: Merci pour votre avis. Nous sommes désolés que vous n'ayez pas apprécié le goût de notre produit. Nous avons une politique de qualité stricte pour nous assurer que seuls des produits frais et authentiques sont vendus. Nous aimerions en savoir plus sur votre expérience d'achat afin que nous puissions en savoir plus sur ce qui s'est passé dans votre cas. N'hésitez pas à nous contacter directement pour discuter davantage."}

## Router Chain


In [138]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [136]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [137]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [139]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['prompt_template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_for_chains, prompt=prompt)
    destination_chains[name] = chain

destination = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destination_str = "\n".join(destination)

In [140]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm_for_chains, prompt=default_prompt)

In [141]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [144]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm=llm_for_chains, prompt=router_prompt)

In [158]:
router_chain.llm_chain.prompt.template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math q

In [145]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True
                         )

In [146]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation is the thermal electromagnetic radiation emitted by a perfect black body (an object that absorbs all incident radiation). It is characterized by a continuous spectrum of wavelengths and is dependent solely on the object\'s temperature, following Planck\'s law. The term "black body" refers to the fact that the object is perfectly black, meaning it absorbs all radiation that falls on it, without reflecting any. Understanding black body radiation is important in fields such as thermodynamics and astrophysics.'

In [147]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can easily answer that the sum of 2 and 2 is equal to 4.'

In [148]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why is DNA important for cell function?'}

ValueError: Received invalid destination chain name 'biology'


这个的用法还是挺高级的，在多分类任务中用的到。比如在数据分析领域，有几个分类的任务：
- 数据查询；（调用查询API）
- 数据处理；（数据处理这一步因为存在数据泄露风险，还是主要以调接口为主）
- 绘图；（调用绘图API，但是如果统一调用API的话，实际上就不是多分类的任务了）

# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

这个功能主要是基于某个大型文档进行问答的，主要的逻辑是将文档进行向量化存储，且有index索引，在文档查询过程中可以快速进行匹配。

不过暂时还用不太上，后面再仔细学习下。

In [3]:
from langchain.chains import RetrievalQA # 可以对文档进行检索
from langchain.document_loaders import CSVLoader

 # 向量存储，这里是用内存中的doc数组搜索向量存储，因为是在内存中的所以不需要链接到任何外部向量数据库，非常容易入门（当然还有其他类型向量存储）
from langchain.vectorstores import DocArrayInMemorySearch

# 向量存储索引创建器，可以帮我们非常容易地创建一个向量存储
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown

In [4]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
loader

In [5]:
index = VectorstoreIndexCreator(
    # 指定向量存储类，这里是 DocArrayInMemorySearch
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])



: 

: 

In [164]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [165]:
response = index.query(query)

NameError: name 'index' is not defined

# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

这部分还没有很好的理解，后期应该会用到，后面回头再来看一下。

## Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()
data[:5]

[Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}),
 Document(page_content=': 1\nname: Recycled Waterhog Dog Mat, Chevron Weave\ndescription: Protect your floors from spil

In [4]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

: 

: 

# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

代理是可以使用各种工具实现更智能流程上的自动化，这部分暂时还用不上。后面有需求再详细了解。